# Lecture 3 – Data 100, Summer 2024

Data 100, Summer 2024

[Acknowledgments Page](https://ds100.org/su24/acks/)

A demonstration of advanced `pandas` syntax to accompany Lecture 3.

In [6]:
import numpy as np
import pandas as pd
import plotly.express as px

In [7]:
weird = pd.DataFrame({
    1:["topdog","botdog"], 
    "1":["topcat","botcat"]
})
weird

,1,1
0,topdog,topcat
1,botdog,botcat


### <font color='red'>STOP!</font> Extraction Exercise

Try figuring out what the following cells evaluate to **without** running them:

In [8]:
weird[1]

0    topdog
1    botdog
Name: 1, dtype: object

In [9]:
weird["1"]

0    topcat
1    botcat
Name: 1, dtype: object

In [10]:
weird[1:]

,1,1
1,botdog,botcat


In [11]:
weird[["1", 1]]

,1,1
0,topcat,topdog
1,botcat,botdog


In [12]:
# Results in a KeyError

weird.loc[:,0]

KeyError: 0

In [13]:
weird.loc[1]

1    botdog
1    botcat
Name: 1, dtype: object

## Dataset: California baby names

In today's lecture, we'll work with the `babynames` dataset, which contains information about the names of infants born in California.

The cell below pulls census data from a government website and then loads it into a usable form. The code shown here is outside of the scope of Data 100, but you're encouraged to dig into it if you are interested!

In [14]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "data/babynamesbystate.zip"
if not os.path.exists(local_filename): # If the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'STATE.CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.head()

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134


In [15]:
babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5


## Conditional Selection

In [16]:
# Ask yourself: Why is :9 is the correct slice to select the first 10 rows?
babynames_first_10_rows = babynames.loc[:9, :]

babynames_first_10_rows

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
5,CA,F,1910,Ruth,128
6,CA,F,1910,Evelyn,126
7,CA,F,1910,Alice,118
8,CA,F,1910,Virginia,101
9,CA,F,1910,Elizabeth,93


By passing in a sequence (list, array, or `Series`) of boolean values, we can extract a subset of the rows in a `DataFrame`. We will keep *only* the rows that correspond to a boolean value of `True`.

In [20]:
# Notice how we have exactly 10 elements in our boolean array argument.
babynames_first_10_rows[[True, False, True, False, True, False, True, False, True, False]]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
2,CA,F,1910,Dorothy,220
4,CA,F,1910,Frances,134
6,CA,F,1910,Evelyn,126
8,CA,F,1910,Virginia,101


In [21]:
# Or using .loc to filter a DataFrame by a Boolean array argument.
babynames_first_10_rows.loc[[True, False, True, False, True, False, True, False, True, False], :]


,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
2,CA,F,1910,Dorothy,220
4,CA,F,1910,Frances,134
6,CA,F,1910,Evelyn,126
8,CA,F,1910,Virginia,101


Oftentimes, we'll use boolean selection to check for entries in a `DataFrame` that meet a particular condition.

In [22]:
# First, use a logical condition to generate a boolean Series
logical_operator = (babynames["Sex"] == "F")
logical_operator

0          True
1          True
2          True
3          True
4          True
          ...  
407423    False
407424    False
407425    False
407426    False
407427    False
Name: Sex, Length: 407428, dtype: bool

In [23]:
# Then, use this boolean Series to filter the DataFrame
babynames[logical_operator]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
239532,CA,F,2022,Zemira,5
239533,CA,F,2022,Ziggy,5
239534,CA,F,2022,Zimal,5
239535,CA,F,2022,Zosia,5


Boolean selection also works with `loc`!

In [24]:
# Notice that we did not have to specify columns to select 
# If no columns are referenced, pandas will automatically select all columns
babynames.loc[babynames["Sex"] == "F"]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
239532,CA,F,2022,Zemira,5
239533,CA,F,2022,Ziggy,5
239534,CA,F,2022,Zimal,5
239535,CA,F,2022,Zosia,5


To filter on multiple conditions, we combine boolean operators using **bitwise comparisons**.

Symbol | Usage      | Meaning 
------ | ---------- | -------------------------------------
~    | ~p       | Returns negation of p
&#124; | p &#124; q | p OR q
&    | p & q    | p AND q
^  | p ^ q | p XOR q (exclusive or)

In [25]:
babynames[(babynames["Sex"] == "F") & (babynames["Year"] < 2000)]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
149050,CA,F,1999,Zareen,5
149051,CA,F,1999,Zeinab,5
149052,CA,F,1999,Zhane,5
149053,CA,F,1999,Zoha,5


In [26]:
babynames[(babynames["Sex"] == "F") | (babynames["Year"] < 2000)]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
342435,CA,M,1999,Yuuki,5
342436,CA,M,1999,Zakariya,5
342437,CA,M,1999,Zavier,5
342438,CA,M,1999,Zayn,5


### <font color='red'>STOP!</font> Slido Exercise

Try answering the Slido poll/following question **without** running the next cell: Which of the following pandas statements returns a DataFrame of the first 3 baby names with `Count > 300`?

<img src="images/slido_1.png" width="200"/>

In [30]:
babynames.iloc[[0, 233, 485], [3, 4]]

,Name,Count
0,Mary,295
233,Mary,390
485,Dorothy,406


In [29]:
babynames.loc[[0, 233, 485]]

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
233,CA,F,1911,Mary,390
485,CA,F,1912,Dorothy,406


In [27]:
babynames.loc[babynames["Count"] > 300, ["Name", "Count"]].head(3)

,Name,Count
233,Mary,390
484,Mary,534
485,Dorothy,406


In [28]:
babynames.loc[babynames["Count"] > 300, ["Name", "Count"]].iloc[0:2, :]

,Name,Count
233,Mary,390
484,Mary,534


In [31]:
# Note: The parentheses surrounding the code make it possible to break the code into multiple lines for readability

(
    babynames[(babynames["Name"] == "Angela") | 
              (babynames["Name"] == "Jacob") |
              (babynames["Name"] == "Zekai") |
              (babynames["Name"] == "Maya")]
)


,State,Sex,Year,Name,Count
137,CA,F,1910,Angela,9
381,CA,F,1911,Angela,10
608,CA,F,1912,Angela,18
936,CA,F,1913,Angela,17
1292,CA,F,1914,Angela,15
...,...,...,...,...,...
393159,CA,M,2018,Jacob,1718
396014,CA,M,2019,Jacob,1664
398882,CA,M,2020,Jacob,1296
401680,CA,M,2021,Jacob,1221


In [32]:
# A more concise method to achieve the above: .isin
names = ["Angela", "Jacob", "Zekai", "Maya"]
display(babynames["Name"].isin(names))
display(babynames[babynames["Name"].isin(names)])

0         False
1         False
2         False
3         False
4         False
          ...  
407423    False
407424    False
407425    False
407426    False
407427    False
Name: Name, Length: 407428, dtype: bool

,State,Sex,Year,Name,Count
137,CA,F,1910,Angela,9
381,CA,F,1911,Angela,10
608,CA,F,1912,Angela,18
936,CA,F,1913,Angela,17
1292,CA,F,1914,Angela,15
...,...,...,...,...,...
393159,CA,M,2018,Jacob,1718
396014,CA,M,2019,Jacob,1664
398882,CA,M,2020,Jacob,1296
401680,CA,M,2021,Jacob,1221


In [33]:
# What if we only want names that start with "M"?
display(babynames["Name"].str.startswith("M"))
display(babynames[babynames["Name"].str.startswith("M")])

0          True
1         False
2         False
3          True
4         False
          ...  
407423    False
407424    False
407425    False
407426    False
407427    False
Name: Name, Length: 407428, dtype: bool

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
3,CA,F,1910,Margaret,163
11,CA,F,1910,Marie,90
12,CA,F,1910,Mildred,90
22,CA,F,1910,Marjorie,60
...,...,...,...,...,...
407308,CA,M,2022,Milano,5
407309,CA,M,2022,Million,5
407310,CA,M,2022,Monty,5
407311,CA,M,2022,Moshe,5


### <font color='red'>STOP!</font> Conditional Selection Exercise

If possible, try answering the following questions without peeking:
* What is the count for `Alex` for `sex` `F` in `2000`?
* How do I get all the rows where `Name` starts with `Kev` or `Count > 600`

In [37]:
babynames[(babynames['Name'] == 'Alex') & (babynames['Sex'] == 'F') & (babynames['Year'] == 2000)].iloc[0, 4]

27

In [38]:
babynames[(babynames["Name"].str.startswith("Kev")) | (babynames['Count'] > 600)]

,State,Sex,Year,Name,Count
1120,CA,F,1914,Mary,773
1488,CA,F,1915,Mary,998
1489,CA,F,1915,Dorothy,717
1490,CA,F,1915,Helen,602
1950,CA,F,1916,Mary,1091
...,...,...,...,...,...
404607,CA,M,2022,Asher,628
404608,CA,M,2022,Adriel,626
404609,CA,M,2022,Luis,607
404610,CA,M,2022,Joshua,605


In [39]:
babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5


## Adding, Removing, and Modifying Columns

To add a column, use `[]` to reference the desired new column, then assign it to a `Series` or array of appropriate length.

In [40]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths

babynames

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,4
1,CA,F,1910,Helen,239,5
2,CA,F,1910,Dorothy,220,7
3,CA,F,1910,Margaret,163,8
4,CA,F,1910,Frances,134,7
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,7
407424,CA,M,2022,Zia,5,3
407425,CA,M,2022,Zora,5,4
407426,CA,M,2022,Zuriel,5,6


To modify a column, use `[]` to access the desired column, then re-assign it to a new array or Series.

In [41]:
# Modify the "name_lengths" column to be one less than its original value
babynames["name_lengths"] = babynames["name_lengths"] - 1
babynames

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,3
1,CA,F,1910,Helen,239,4
2,CA,F,1910,Dorothy,220,6
3,CA,F,1910,Margaret,163,7
4,CA,F,1910,Frances,134,6
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,6
407424,CA,M,2022,Zia,5,2
407425,CA,M,2022,Zora,5,3
407426,CA,M,2022,Zuriel,5,5


Rename a column using the `.rename()` method.

In [42]:
# Rename "name_lengths" to "Length"
babynames = babynames.rename(columns={"name_lengths":"Length"})
babynames

,State,Sex,Year,Name,Count,Length
0,CA,F,1910,Mary,295,3
1,CA,F,1910,Helen,239,4
2,CA,F,1910,Dorothy,220,6
3,CA,F,1910,Margaret,163,7
4,CA,F,1910,Frances,134,6
...,...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5,6
407424,CA,M,2022,Zia,5,2
407425,CA,M,2022,Zora,5,3
407426,CA,M,2022,Zuriel,5,5


Remove a column using `.drop()`.

In [43]:
# Remove our new "Length" column
babynames = babynames.drop("Length", axis="columns")
babynames

,State,Sex,Year,Name,Count
0,CA,F,1910,Mary,295
1,CA,F,1910,Helen,239
2,CA,F,1910,Dorothy,220
3,CA,F,1910,Margaret,163
4,CA,F,1910,Frances,134
...,...,...,...,...,...
407423,CA,M,2022,Zayvier,5
407424,CA,M,2022,Zia,5
407425,CA,M,2022,Zora,5
407426,CA,M,2022,Zuriel,5


## Useful Utility Functions

#### `NumPy`

The `NumPy` functions you encountered in [Data 8](https://www.data8.org/su23/reference/#array-functions-and-methods) are compatible with objects in `pandas`. 

In [44]:
yash_counts = babynames[babynames["Name"] == "Yash"]["Count"]
yash_counts

331824     8
334114     9
336390    11
338773    12
341387    10
343571    14
345767    24
348230    29
350889    24
353445    29
356221    25
358978    27
361831    29
364905    24
367867    23
370945    18
374055    14
376756    18
379660    18
383338     9
385903    12
388529    17
391485    16
394906    10
397874     9
400171    15
403092    13
406006    13
Name: Count, dtype: int64

In [45]:
# Average number of babies named Yash each year

np.mean(yash_counts)

17.142857142857142

In [46]:
# Max number of babies named Yash born in any single year

max(yash_counts)

29

#### Built-In `pandas` Methods

There are many, *many* utility functions built into `pandas`, far more than we can possibly cover in lecture. You are encouraged to explore all the functionality outlined in the `pandas` [documentation](https://pandas.pydata.org/docs/reference/index.html).

In [47]:
# Returns the shape of the object in the format (num_rows, num_columns)
babynames.shape

(407428, 5)

In [48]:
# Returns the total number of entries in the object, equal to num_rows * num_columns
babynames.size

2037140

In [49]:
# What summary statistics can we describe?
babynames.describe()

,Year,Count
count,407428.000000,407428.000000
mean,1985.733609,79.543456
std,27.007660,293.698654
min,1910.000000,5.000000
25%,1969.000000,7.000000
50%,1992.000000,13.000000
75%,2008.000000,38.000000
max,2022.000000,8260.000000


In [50]:
# Our statistics are slightly different when working with a Series
babynames["Sex"].describe()

count     407428
unique         2
top            F
freq      239537
Name: Sex, dtype: object

In [51]:
# Randomly sample row(s) from the DataFrame
babynames.sample()

,State,Sex,Year,Name,Count
119420,CA,F,1991,Latanya,6


In [54]:
# Rerun this cell a few times – you'll get different results!
babynames.sample(5).iloc[:, 2:]

,Year,Name,Count
177515,2007,Celia,48
41311,1959,Cristy,13
73774,1976,Susana,227
176764,2007,Ariana,800
93830,1983,Nida,5


In [58]:
# Sampling with replacement
babynames[babynames["Year"] == 2000].sample(4, replace = True).iloc[:,2:]

,Year,Name,Count
149600,2000,Lilly,67
343158,2000,Jaylin,28
151899,2000,Vivianne,7
342776,2000,Octavio,111


In [59]:
# Count the number of times each unique value occurs in a Series
babynames["Name"].value_counts()

Name
Jean         223
Francis      221
Guadalupe    218
Jessie       217
Marion       214
            ... 
Renesme        1
Purity         1
Olanna         1
Nohea          1
Zayvier        1
Name: count, Length: 20437, dtype: int64

In [60]:
# Return an array of all unique values in the Series
babynames["Name"].unique()

array(['Mary', 'Helen', 'Dorothy', ..., 'Zae', 'Zai', 'Zayvier'],
      dtype=object)

In [61]:
# Sort a Series
babynames["Name"].sort_values()

366001      Aadan
384005      Aadan
369120      Aadan
398211    Aadarsh
370306      Aaden
           ...   
220691      Zyrah
197529      Zyrah
217429      Zyrah
232167      Zyrah
404544      Zyrus
Name: Name, Length: 407428, dtype: object

In [62]:
# Sort a DataFrame – there are lots of Michaels in California
babynames.sort_values(by="Count", ascending=False)

,State,Sex,Year,Name,Count
268041,CA,M,1957,Michael,8260
267017,CA,M,1956,Michael,8258
317387,CA,M,1990,Michael,8246
281850,CA,M,1969,Michael,8245
283146,CA,M,1970,Michael,8196
...,...,...,...,...,...
317292,CA,M,1989,Olegario,5
317291,CA,M,1989,Norbert,5
317290,CA,M,1989,Niles,5
317289,CA,M,1989,Nikola,5


## Custom sorting

### Approach 1: Create a temporary column

In [63]:
# Create a Series of the length of each name
babyname_lengths = babynames["Name"].str.len()

# Add a column named "name_lengths" that includes the length of each name
babynames["name_lengths"] = babyname_lengths
babynames.head(5)

,State,Sex,Year,Name,Count,name_lengths
0,CA,F,1910,Mary,295,4
1,CA,F,1910,Helen,239,5
2,CA,F,1910,Dorothy,220,7
3,CA,F,1910,Margaret,163,8
4,CA,F,1910,Frances,134,7


In [64]:
# Sort by the temporary column
babynames = babynames.sort_values(by="name_lengths", ascending=False)
babynames.head(5)

,State,Sex,Year,Name,Count,name_lengths
334166,CA,M,1996,Franciscojavier,8,15
337301,CA,M,1997,Franciscojavier,5,15
339472,CA,M,1998,Franciscojavier,6,15
321792,CA,M,1991,Ryanchristopher,7,15
327358,CA,M,1993,Johnchristopher,5,15


In [65]:
# Drop the 'name_length' column
babynames = babynames.drop("name_lengths", axis="columns")
babynames.head(5)

,State,Sex,Year,Name,Count
334166,CA,M,1996,Franciscojavier,8
337301,CA,M,1997,Franciscojavier,5
339472,CA,M,1998,Franciscojavier,6
321792,CA,M,1991,Ryanchristopher,7
327358,CA,M,1993,Johnchristopher,5


### Approach 2: Sorting using the `key` argument

In [66]:
babynames.sort_values("Name", key=lambda x:x.str.len(), ascending=False).head()

,State,Sex,Year,Name,Count
334166,CA,M,1996,Franciscojavier,8
327472,CA,M,1993,Ryanchristopher,5
337301,CA,M,1997,Franciscojavier,5
337477,CA,M,1997,Ryanchristopher,5
312543,CA,M,1987,Franciscojavier,5


### Approach 3: Sorting Using the `map` Function

We can also use the Python map function if we want to use an arbitrarily defined function. Suppose we want to sort by the number of occurrences of "dr" plus the number of occurences of "ea".

In [67]:
# First, define a function to count the number of times "dr" or "ea" appear in each name
def dr_ea_count(string):
    return string.count('dr') + string.count('ea')

# Then, use `map` to apply `dr_ea_count` to each name in the "Name" column
babynames["dr_ea_count"] = babynames["Name"].map(dr_ea_count)

# Sort the DataFrame by the new "dr_ea_count" column so we can see our handiwork
babynames = babynames.sort_values(by="dr_ea_count", ascending=False)
babynames.head()

,State,Sex,Year,Name,Count,dr_ea_count
115957,CA,F,1990,Deandrea,5,3
101976,CA,F,1986,Deandrea,6,3
131029,CA,F,1994,Leandrea,5,3
108731,CA,F,1988,Deandrea,5,3
308131,CA,M,1985,Deandrea,6,3


In [68]:
# Drop the `dr_ea_count` column
babynames = babynames.drop("dr_ea_count", axis="columns")
babynames.head(5)

,State,Sex,Year,Name,Count
115957,CA,F,1990,Deandrea,5
101976,CA,F,1986,Deandrea,6
131029,CA,F,1994,Leandrea,5
108731,CA,F,1988,Deandrea,5
308131,CA,M,1985,Deandrea,6


## Grouping

Group rows that share a common feature, then aggregate data across the group.

In this example, we count the total number of babies born in each year (considering only a small subset of the data, for simplicity).

<img src="images/groupby.png" width="800"/>

In [70]:
# The code below uses the full babynames dataset, which is why some numbers are different relative to the diagram
babynames[["Year", "Count"]].groupby("Year").sum()

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2018,395436
2019,386996
2020,362882


There are many different aggregation functions we can use, all of which are useful in different applications.

In [72]:
# What is the earliest year in which each name appeared?
babynames.groupby("Name")[["Year"]].min()

,Year
Name,
Aadan,2008
Aadarsh,2019
Aaden,2007
Aadhav,2014
Aadhini,2022
...,...
Zymir,2020
Zyon,1999
Zyra,2012


In [73]:
# What is the largest single-year count of each name?
babynames.groupby("Name")[["Count"]].max()

,Count
Name,
Aadan,7
Aadarsh,6
Aaden,158
Aadhav,8
Aadhini,6
...,...
Zymir,5
Zyon,17
Zyra,16


In this example, we count the total number of babies born each year (considering only a small subset of the data for simplicity).

In [74]:
babynames.groupby("Year")

In [75]:
# Selecting only numerical columns to perform grouping on and then grouping by "Year"
babies_by_year = babynames[["Year", "Count"]].groupby("Year").sum()
babies_by_year

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2018,395436
2019,386996
2020,362882


What happens if we don't select columns `Year` and `Count` before calling `groupby` and our aggregation function? The results are messy! 

In [76]:
babynames.groupby("Year").sum()

,State,Sex,Name,Count
Year,,,,
1910,CACACACACACACACACACACACACACACACACACACACACACACA...,FFFFFFFFFFFMMFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,JeannetteBeatriceJeanetteAdrienneJeanneAltheaA...,9163
1911,CACACACACACACACACACACACACACACACACACACACACACACA...,FFFMFFFFFFFFMFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,AdrienneDorotheaJeanettePedroJeannetteBeatrice...,9983
1912,CACACACACACACACACACACACACACACACACACACACACACACA...,MFFFFFMMFFFFFMFFMFFFFFFFFFFFFFFFFFFFFFFFFFFFFF...,PedroBeatriceDorotheaEleanoreJeannetteJeanette...,17946
1913,CACACACACACACACACACACACACACACACACACACACACACACA...,FMMFFMFFFFMFFMFFFMMFFFFFFFFFFFFFFFFFFFFFFFFFFF...,JeannetteJeanDeanBeatriceEleanorePedroJeanette...,22094
1914,CACACACACACACACACACACACACACACACACACACACACACACA...,FMMFFFFMFFMFFFMFFFMMFFFFFFFFFFFFFFFFFFFFFFFFFF...,JeannettePedroDeanDorotheaJeanetteBeatriceElea...,26926
...,...,...,...,...
2018,CACACACACACACACACACACACACACACACACACACACACACACA...,MMFFMMMMFFFFFFMMMMFFFFMMFMFFFMFMMFMMMFFMFMFMFF...,DeandreAndreasLeandraAndreaLeandroDeaconIdrisM...,395436
2019,CACACACACACACACACACACACACACACACACACACACACACACA...,FMFMMFFFMFFMFMFMMMFMMMMFFMMMMMMFFFMMMMFMFFMFMF...,AlexandreaAndreasAndreaDeandreLeandroAureaSidr...,386996
2020,CACACACACACACACACACACACACACACACACACACACACACACA...,MFMMMFMFFFFFFFMMMMMMMMMMMMFMMMMMMFMMMFMMFFFFFF...,LeandroAndreaDeandreAndreasLeandreAlexandreaHe...,362882


Alternatively, we could select the relevant columns after calling `groupby` from the "sub-`DataFrames`":

In [77]:
babynames.groupby("Year")[["Year", "Count"]].sum()

,Year,Count
Year,,
1910,693330,9163
1911,751023,9983
1912,1066896,17946
1913,1172669,22094
1914,1358940,26926
...,...,...
2018,13219918,395436
2019,13171956,386996
2020,12956280,362882


Or, another way (Note: the result is slightly different as it doesn't aggregate the `Year` column despite it being numeric because we are grouping by it): 

In [78]:
babynames.groupby("Year").sum(numeric_only=True)

,Count
Year,
1910,9163
1911,9983
1912,17946
1913,22094
1914,26926
...,...
2018,395436
2019,386996
2020,362882


In [79]:
# Plotting baby counts per year
fig = px.line(babies_by_year, y = "Count")
fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=700, 
                  height=400)

### <font color='red'>STOP!</font> Slido Exercise

Try answering the Slido poll/following question **without** looking at the next image. Try to predict the results of the `groupby` operation shown. 

The answer is below the image.

<img src="images/slido_groupby.png" alt="Image" width="600">

The top ?? will be "hi", the second ?? will be "tx", and the third ?? will be "sd". 

In [80]:
ds = pd.DataFrame(dict(x=[3, 1, 4, 1, 5, 9, 2, 5, 6], 
                      y=['ak', 'tx', 'fl', 'hi', 'mi', 'ak', 'ca', 'sd', 'nc']), 
                      index=list('ABCABCACB') )
ds

,x,y
A,3,ak
B,1,tx
C,4,fl
A,1,hi
B,5,mi
C,9,ak
A,2,ca
C,5,sd
B,6,nc


In [82]:
# Performing groupby on the first column with max aggregation function
ds.groupby(ds.index).max()

,x,y
A,3,hi
B,6,tx
C,9,sd


In [83]:
('hi' > 'ak') & ('hi' > 'ca')

True

***
If we have extra time.... Otherwise this will be next lecture!

### Case Study: Name "Popularity"

In this exercise, let's find the name with sex "F" that has dropped most in popularity since its peak usage in California. We'll start by filtering `babynames` to only include names corresponding to sex "F".

In [84]:
f_babynames = babynames[babynames["Sex"] == "F"]
f_babynames

,State,Sex,Year,Name,Count
115957,CA,F,1990,Deandrea,5
101976,CA,F,1986,Deandrea,6
131029,CA,F,1994,Leandrea,5
108731,CA,F,1988,Deandrea,5
193104,CA,F,2010,Deandra,5
...,...,...,...,...,...
192320,CA,F,2010,Annaly,7
212793,CA,F,2015,Iyanna,6
202350,CA,F,2013,Lailah,50
212784,CA,F,2015,Helene,6


In [85]:
# We sort the data by "Year"
f_babynames = f_babynames.sort_values("Year")
f_babynames

,State,Sex,Year,Name,Count
23,CA,F,1910,Bernice,59
219,CA,F,1910,Katharine,5
170,CA,F,1910,Clarice,7
26,CA,F,1910,Doris,56
30,CA,F,1910,Ethel,52
...,...,...,...,...,...
236551,CA,F,2022,Dani,47
235914,CA,F,2022,Iris,361
235918,CA,F,2022,Lucy,345
235973,CA,F,2022,Sage,226


To build our intuition on how to answer our research question, let's visualize the prevalence of the name "Jennifer" over time.

In [86]:
# We'll talk about how to generate plots in a later lecture
fig = px.line(f_babynames[f_babynames["Name"] == "Jennifer"],
              x="Year", y="Count")

fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)

We'll need a mathematical definition for the change in popularity of a name in California.

Define the metric "Ratio to Peak" (RTP). We'll calculate this as the count of the name in 2022 (the most recent year for which we have data) divided by the largest count of this name in *any* year. 

A demo calculation for Jennifer:

In [87]:
# In the year with the highest Jennifer count, 6065 Jennifers were born
max_jenn = np.max(f_babynames[f_babynames["Name"] == "Jennifer"]["Count"])
max_jenn

6065

In [88]:
# Remember that we sorted f_babynames by "Year". 
# This means that grabbing the final entry gives us the most recent count of Jennifers: 114
# In 2022, the most recent year for which we have data, 114 Jennifers were born
curr_jenn = f_babynames[f_babynames["Name"] == "Jennifer"]["Count"].iloc[-1]
curr_jenn

114

In [89]:
# Compute the RTP
curr_jenn / max_jenn

0.018796372629843364

We can also write a function that produces the `ratio_to_peak`for a given `Series`. This will allow us to use `.groupby` to speed up our computation for all names in the dataset.

In [90]:
def ratio_to_peak(series):
    """
    Compute the RTP for a Series containing the counts per year for a single name (year column sorted ascendingly).
    """
    return series.iloc[-1] / np.max(series)

In [91]:
# Construct a Series containing our Jennifer count data
jenn_counts_ser = f_babynames[f_babynames["Name"] == "Jennifer"]["Count"]

# Then, find the RTP
ratio_to_peak(jenn_counts_ser)

0.018796372629843364

Now, let's use `.groupby` to compute the RTPs for *all* names in the dataset.

You may see a warning message when running the cell below. As discussed in the lecture, `pandas` can't apply an aggregation function to non-numeric data (it doens't make sense to divide "CA" by a number). We can select numerical columns of interest directly.

In [92]:
# Results in a TypeError
rtp_table = f_babynames.groupby("Name").agg(ratio_to_peak)
rtp_table

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [96]:
rtp_table = f_babynames.groupby("Name")[["Year", "Count"]].agg(ratio_to_peak)
rtp_table

,Year,Count
Name,,
Aadhini,1.0,1.000000
Aadhira,1.0,0.500000
Aadhya,1.0,0.660000
Aadya,1.0,0.586207
Aahana,1.0,0.269231
...,...,...
Zyanya,1.0,0.466667
Zyla,1.0,1.000000
Zylah,1.0,1.000000


This is the `pandas` equivalent of `.group` from [Data 8](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html). If we wanted to achieve this same result using the `datascience` library, we would write:

`f_babynames.group("Name", ratio_to_peak)`

### <font color='red'>STOP!</font> Slido Exercise

Try answering the Slido poll/following question **without** running the next cell: Is there a row where `Year` is not equal to 1?

In [97]:
# Unique values in the Year column
rtp_table["Year"].unique()

array([1.])

In [98]:
# Dropping the "Year" column
rtp_table.drop("Year", axis="columns", inplace=True)
rtp_table

,Count
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.660000
Aadya,0.586207
Aahana,0.269231
...,...
Zyanya,0.466667
Zyla,1.000000
Zylah,1.000000


In [99]:
# Rename "Count" to "Count RTP" for clarity
rtp_table = rtp_table.rename(columns = {"Count": "Count RTP"})
rtp_table

,Count RTP
Name,
Aadhini,1.000000
Aadhira,0.500000
Aadhya,0.660000
Aadya,0.586207
Aahana,0.269231
...,...
Zyanya,0.466667
Zyla,1.000000
Zylah,1.000000


In [100]:
# What name has fallen the most in popularity?
rtp_table.sort_values("Count RTP")

,Count RTP
Name,
Debra,0.001260
Debbie,0.002815
Carol,0.003180
Tammy,0.003249
Susan,0.003305
...,...
Fidelia,1.000000
Naveyah,1.000000
Finlee,1.000000


We can visualize the decrease in the popularity of the name "Debra:"

In [101]:
def plot_name(*names):
    fig = px.line(f_babynames[f_babynames["Name"].isin(names)], 
                  x = "Year", y = "Count", color="Name",
                  title=f"Popularity for: {names}")
    fig.update_layout(font_size = 18, 
                  autosize=False, 
                  width=1000, 
                  height=400)
    return fig

plot_name("Debra")

In [102]:
# Find the 10 names that have decreased the most in popularity
top10 = rtp_table.sort_values("Count RTP").head(10).index
top10

Index(['Debra', 'Debbie', 'Carol', 'Tammy', 'Susan', 'Cheryl', 'Shannon',
       'Tina', 'Michele', 'Terri'],
      dtype='object', name='Name')

In [103]:
plot_name(*top10)